In [1]:
import os

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
from torchvision import models


In [3]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

In [4]:
TOTAL_CHAR_COUNT = 26 + 10
TOTAL_CAPTCHA_LEN = 5
CHAR_SET = []
for num in range(10):
    CHAR_SET.append(str(num))
for chari in range(97, 97+26):
    CHAR_SET.append(chr(chari))

In [5]:
def getOneHotEncoding(s):
    ohev = torch.zeros(TOTAL_CAPTCHA_LEN*TOTAL_CHAR_COUNT, dtype = torch.float32)
    for ind in range(TOTAL_CAPTCHA_LEN):
        c = s[ind]
        ohev[TOTAL_CHAR_COUNT*ind + CHAR_SET.index(c)] = 1
    return ohev

In [6]:
class CaptchaDataset(Dataset):
    def __init__(self, img_path, transform=None):
        self.img_path = img_path
        self.img_list = os.listdir(self.img_path)
        self.transform = transform
        
    def __getitem__(self, idx):
        img_name = self.img_list[idx]
        img = Image.open(self.img_path +'/'+ img_name)
        img = img.convert('L')
        label = (img_name)[:-4]
        #print(label)
        ohev = getOneHotEncoding(label)
        if self.transform is not None:
            img = self.transform(img)
        return img, ohev, label
    
    def __len__(self):
        return len(self.img_list)

In [7]:
IMG_H, IMG_W = 224, 224

In [8]:
transform = transforms.Compose([
    transforms.Resize([IMG_H, IMG_W]),
    transforms.ToTensor(),
])

In [9]:
train_ds = CaptchaDataset('./CaptchaDataset', transform=transform)
test_ds = CaptchaDataset('./Captcha_Dataset', transform = transform)
train_dl = DataLoader(train_ds, batch_size=64, num_workers=4)
test_dl = DataLoader(test_ds, batch_size=64, num_workers=4)

In [10]:
# #model = models.DenseNet(growth_rate=32, block_config=(2, 4, 12, 8), num_classes = TOTAL_CAPTCHA_LEN*TOTAL_CHAR_COUNT)
model = models.resnet18(pretrained=False)

In [11]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv0 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=True)
        self.fcl = nn.Linear(in_features = 512,out_features=TOTAL_CAPTCHA_LEN*TOTAL_CHAR_COUNT, bias=True)
        self.sigmoid = nn.Sigmoid()
        self.model = models.resnet18(pretrained = False)
        self.model = nn.Sequential(*list(model.children())[1:-1])
    def forward(self, x):
        out = self.conv0(x)
        out = self.model(out)
        #print(out.shape)
        out = out.reshape(out.size(0), -1)
        #print(out.shape)
        out = self.fcl(out)
        out = self.sigmoid(out)
        return out

In [12]:
# conv0 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=True)
# fcl = nn.Linear(in_features = 32768,out_features=TOTAL_CAPTCHA_LEN*TOTAL_CHAR_COUNT, bias=True)
# new_model = nn.Sequential(conv0, *list(model.children())[1:-1], fcl)

In [13]:
new_model = ResNet()

In [14]:
loss_func = nn.MultiLabelSoftMarginLoss()
optm = torch.optim.Adam(new_model.parameters(), lr=0.0001)

In [15]:
new_model.cuda()

ResNet(
  (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (fcl): Linear(in_features=512, out_features=180, bias=True)
  (sigmoid): Sigmoid()
  (model): Sequential(
    (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (3): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, e

In [ ]:
for epoch in range(30):
    for step, i in enumerate(train_dl):
        img, ohev, label = i
        img = Variable(img).cuda()
        ohev = Variable(ohev.float()).cuda()
        pred = new_model(img)
        loss = loss_func(pred, ohev)
        optm.zero_grad()
        loss.backward()
        optm.step()
        print('epoch:', epoch+1, 'step:', step+1, 'loss:', loss.item())

epoch: 1 step: 1 loss: 0.9657447338104248
epoch: 1 step: 2 loss: 0.9563207626342773
epoch: 1 step: 3 loss: 0.9480096697807312
epoch: 1 step: 4 loss: 0.939012885093689
epoch: 1 step: 5 loss: 0.9301823377609253
epoch: 1 step: 6 loss: 0.9213101267814636
epoch: 1 step: 7 loss: 0.912197470664978
epoch: 1 step: 8 loss: 0.9034605026245117
epoch: 1 step: 9 loss: 0.895052969455719
epoch: 1 step: 10 loss: 0.8869868516921997
epoch: 1 step: 11 loss: 0.8794848918914795
epoch: 1 step: 12 loss: 0.8719866275787354
epoch: 1 step: 13 loss: 0.8640562295913696
epoch: 1 step: 14 loss: 0.8570289611816406
epoch: 1 step: 15 loss: 0.8502120971679688
epoch: 1 step: 16 loss: 0.8437094688415527
epoch: 1 step: 17 loss: 0.8368910551071167
epoch: 1 step: 18 loss: 0.8307280540466309
epoch: 1 step: 19 loss: 0.8246472477912903
epoch: 1 step: 20 loss: 0.8187631368637085
epoch: 1 step: 21 loss: 0.8134739398956299
epoch: 1 step: 22 loss: 0.807634711265564
epoch: 1 step: 23 loss: 0.802193284034729
epoch: 1 step: 24 loss: 0

epoch: 1 step: 191 loss: 0.6959306001663208
epoch: 1 step: 192 loss: 0.6959131956100464
epoch: 1 step: 193 loss: 0.6958845257759094
epoch: 1 step: 194 loss: 0.695857048034668
epoch: 1 step: 195 loss: 0.6958396434783936
epoch: 1 step: 196 loss: 0.6958187818527222
epoch: 1 step: 197 loss: 0.6957980394363403
epoch: 1 step: 198 loss: 0.6957738399505615
epoch: 1 step: 199 loss: 0.6957542300224304
epoch: 1 step: 200 loss: 0.6957309246063232
epoch: 1 step: 201 loss: 0.6957130432128906
epoch: 1 step: 202 loss: 0.695692777633667
epoch: 1 step: 203 loss: 0.6956695914268494
epoch: 1 step: 204 loss: 0.6956502199172974
epoch: 1 step: 205 loss: 0.6956386566162109
epoch: 1 step: 206 loss: 0.6956139802932739
epoch: 1 step: 207 loss: 0.6955956220626831
epoch: 1 step: 208 loss: 0.695579469203949
epoch: 1 step: 209 loss: 0.695555567741394
epoch: 1 step: 210 loss: 0.6955392360687256
epoch: 1 step: 211 loss: 0.6955244541168213
epoch: 1 step: 212 loss: 0.6955043077468872
epoch: 1 step: 213 loss: 0.695488035

epoch: 1 step: 379 loss: 0.6940817832946777
epoch: 1 step: 380 loss: 0.694077730178833
epoch: 1 step: 381 loss: 0.6940751671791077
epoch: 1 step: 382 loss: 0.6940714716911316
epoch: 1 step: 383 loss: 0.6940661668777466
epoch: 1 step: 384 loss: 0.6940627098083496
epoch: 1 step: 385 loss: 0.6940588355064392
epoch: 1 step: 386 loss: 0.6940553188323975
epoch: 1 step: 387 loss: 0.694051206111908
epoch: 1 step: 388 loss: 0.6940478086471558
epoch: 1 step: 389 loss: 0.694044828414917
epoch: 1 step: 390 loss: 0.6940399408340454
epoch: 1 step: 391 loss: 0.6940362453460693
epoch: 1 step: 392 loss: 0.694034218788147
epoch: 1 step: 393 loss: 0.6940310001373291
epoch: 1 step: 394 loss: 0.6940263509750366
epoch: 1 step: 395 loss: 0.6940244436264038
epoch: 1 step: 396 loss: 0.6940188407897949
epoch: 1 step: 397 loss: 0.6940162777900696
epoch: 1 step: 398 loss: 0.6940112709999084
epoch: 1 step: 399 loss: 0.6940077543258667
epoch: 1 step: 400 loss: 0.6940048933029175
epoch: 1 step: 401 loss: 0.694000840

epoch: 1 step: 567 loss: 0.6936357617378235
epoch: 1 step: 568 loss: 0.6936349868774414
epoch: 1 step: 569 loss: 0.6936330199241638
epoch: 1 step: 570 loss: 0.6936326026916504
epoch: 1 step: 571 loss: 0.6936309337615967
epoch: 1 step: 572 loss: 0.6936286687850952
epoch: 1 step: 573 loss: 0.6936275362968445
epoch: 1 step: 574 loss: 0.6936261057853699
epoch: 1 step: 575 loss: 0.6936255693435669
epoch: 1 step: 576 loss: 0.6936228275299072
epoch: 1 step: 577 loss: 0.6936219334602356
epoch: 1 step: 578 loss: 0.6936208605766296
epoch: 1 step: 579 loss: 0.6936203241348267
epoch: 1 step: 580 loss: 0.6936181783676147
epoch: 1 step: 581 loss: 0.6936172246932983
epoch: 1 step: 582 loss: 0.6936159133911133
epoch: 1 step: 583 loss: 0.6936147212982178
epoch: 1 step: 584 loss: 0.6936125755310059
epoch: 1 step: 585 loss: 0.6936126351356506
epoch: 1 step: 586 loss: 0.6936113238334656
epoch: 1 step: 587 loss: 0.6936095356941223
epoch: 1 step: 588 loss: 0.6936085820198059
epoch: 1 step: 589 loss: 0.69360

epoch: 1 step: 755 loss: 0.6934524774551392
epoch: 1 step: 756 loss: 0.6934516429901123
epoch: 1 step: 757 loss: 0.6934508085250854
epoch: 1 step: 758 loss: 0.6934508681297302
epoch: 1 step: 759 loss: 0.693449854850769
epoch: 1 step: 760 loss: 0.6934487819671631
epoch: 1 step: 761 loss: 0.6934487819671631
epoch: 1 step: 762 loss: 0.6934480667114258
epoch: 1 step: 763 loss: 0.6934472322463989
epoch: 1 step: 764 loss: 0.6934466361999512
epoch: 1 step: 765 loss: 0.6934459209442139
epoch: 1 step: 766 loss: 0.6934452652931213
epoch: 1 step: 767 loss: 0.6934444904327393
epoch: 1 step: 768 loss: 0.693444013595581
epoch: 1 step: 769 loss: 0.6934434771537781
epoch: 1 step: 770 loss: 0.6934429407119751
epoch: 1 step: 771 loss: 0.6934418678283691
epoch: 1 step: 772 loss: 0.69344162940979
epoch: 1 step: 773 loss: 0.6934412121772766
epoch: 1 step: 774 loss: 0.6934404373168945
epoch: 1 step: 775 loss: 0.6934396028518677
epoch: 1 step: 776 loss: 0.693438708782196
epoch: 1 step: 777 loss: 0.6934387087

...

epoch: 24 step: 65 loss: 0.6931471824645996
epoch: 24 step: 66 loss: 0.6931471824645996
epoch: 24 step: 67 loss: 0.6931471824645996
epoch: 24 step: 68 loss: 0.6931471824645996
epoch: 24 step: 69 loss: 0.6931471824645996
epoch: 24 step: 70 loss: 0.6931471824645996
epoch: 24 step: 71 loss: 0.6931471824645996
epoch: 24 step: 72 loss: 0.6931471824645996
epoch: 24 step: 73 loss: 0.6931471824645996
epoch: 24 step: 74 loss: 0.6931471824645996
epoch: 24 step: 75 loss: 0.6931471824645996
epoch: 24 step: 76 loss: 0.6931471824645996
epoch: 24 step: 77 loss: 0.6931471824645996
epoch: 24 step: 78 loss: 0.6931471824645996
epoch: 24 step: 79 loss: 0.6931471824645996
epoch: 24 step: 80 loss: 0.6931471824645996
epoch: 24 step: 81 loss: 0.6931471824645996
epoch: 24 step: 82 loss: 0.6931471824645996
epoch: 24 step: 83 loss: 0.6931471824645996
epoch: 24 step: 84 loss: 0.6931471824645996
epoch: 24 step: 85 loss: 0.6931471824645996
epoch: 24 step: 86 loss: 0.6931471824645996
epoch: 24 step: 87 loss: 0.69314

epoch: 24 step: 248 loss: 0.6931471824645996
epoch: 24 step: 249 loss: 0.6931471824645996
epoch: 24 step: 250 loss: 0.6931471824645996
epoch: 24 step: 251 loss: 0.6931471824645996
epoch: 24 step: 252 loss: 0.6931471824645996
epoch: 24 step: 253 loss: 0.6931471824645996
epoch: 24 step: 254 loss: 0.6931471824645996
epoch: 24 step: 255 loss: 0.6931471824645996
epoch: 24 step: 256 loss: 0.6931471824645996
epoch: 24 step: 257 loss: 0.6931471824645996
epoch: 24 step: 258 loss: 0.6931471824645996
epoch: 24 step: 259 loss: 0.6931471824645996
epoch: 24 step: 260 loss: 0.6931471824645996
epoch: 24 step: 261 loss: 0.6931471824645996
epoch: 24 step: 262 loss: 0.6931471824645996
epoch: 24 step: 263 loss: 0.6931471824645996
epoch: 24 step: 264 loss: 0.6931471824645996
epoch: 24 step: 265 loss: 0.6931471824645996
epoch: 24 step: 266 loss: 0.6931471824645996
epoch: 24 step: 267 loss: 0.6931471824645996
epoch: 24 step: 268 loss: 0.6931471824645996
epoch: 24 step: 269 loss: 0.6931471824645996
epoch: 24 

epoch: 24 step: 432 loss: 0.6931471824645996
epoch: 24 step: 433 loss: 0.6931471824645996
epoch: 24 step: 434 loss: 0.6931471824645996
epoch: 24 step: 435 loss: 0.6931471824645996
epoch: 24 step: 436 loss: 0.6931471824645996
epoch: 24 step: 437 loss: 0.6931471824645996
epoch: 24 step: 438 loss: 0.6931471824645996
epoch: 24 step: 439 loss: 0.6931471824645996
epoch: 24 step: 440 loss: 0.6931471824645996
epoch: 24 step: 441 loss: 0.6931471824645996
epoch: 24 step: 442 loss: 0.6931471824645996
epoch: 24 step: 443 loss: 0.6931471824645996
epoch: 24 step: 444 loss: 0.6931471824645996
epoch: 24 step: 445 loss: 0.6931471824645996
epoch: 24 step: 446 loss: 0.6931471824645996
epoch: 24 step: 447 loss: 0.6931471824645996
epoch: 24 step: 448 loss: 0.6931471824645996
epoch: 24 step: 449 loss: 0.6931471824645996
epoch: 24 step: 450 loss: 0.6931471824645996
epoch: 24 step: 451 loss: 0.6931471824645996
epoch: 24 step: 452 loss: 0.6931471824645996
epoch: 24 step: 453 loss: 0.6931471824645996
epoch: 24 

epoch: 24 step: 616 loss: 0.6931471824645996
epoch: 24 step: 617 loss: 0.6931471824645996
epoch: 24 step: 618 loss: 0.6931471824645996
epoch: 24 step: 619 loss: 0.6931471824645996
epoch: 24 step: 620 loss: 0.6931471824645996
epoch: 24 step: 621 loss: 0.6931471824645996
epoch: 24 step: 622 loss: 0.6931471824645996
epoch: 24 step: 623 loss: 0.6931471824645996
epoch: 24 step: 624 loss: 0.6931471824645996
epoch: 24 step: 625 loss: 0.6931471824645996
epoch: 24 step: 626 loss: 0.6931471824645996
epoch: 24 step: 627 loss: 0.6931471824645996
epoch: 24 step: 628 loss: 0.6931471824645996
epoch: 24 step: 629 loss: 0.6931471824645996
epoch: 24 step: 630 loss: 0.6931471824645996
epoch: 24 step: 631 loss: 0.6931471824645996
epoch: 24 step: 632 loss: 0.6931471824645996
epoch: 24 step: 633 loss: 0.6931471824645996
epoch: 24 step: 634 loss: 0.6931471824645996
epoch: 24 step: 635 loss: 0.6931471824645996
epoch: 24 step: 636 loss: 0.6931471824645996
epoch: 24 step: 637 loss: 0.6931471824645996
epoch: 24 

epoch: 25 step: 19 loss: 0.6931471824645996
epoch: 25 step: 20 loss: 0.6931471824645996
epoch: 25 step: 21 loss: 0.6931471824645996
epoch: 25 step: 22 loss: 0.6931471824645996
epoch: 25 step: 23 loss: 0.6931471824645996
epoch: 25 step: 24 loss: 0.6931471824645996
epoch: 25 step: 25 loss: 0.6931471824645996
epoch: 25 step: 26 loss: 0.6931471824645996
epoch: 25 step: 27 loss: 0.6931471824645996
epoch: 25 step: 28 loss: 0.6931471824645996
epoch: 25 step: 29 loss: 0.6931471824645996
epoch: 25 step: 30 loss: 0.6931471824645996
epoch: 25 step: 31 loss: 0.6931471824645996
epoch: 25 step: 32 loss: 0.6931471824645996
epoch: 25 step: 33 loss: 0.6931471824645996
epoch: 25 step: 34 loss: 0.6931471824645996
epoch: 25 step: 35 loss: 0.6931471824645996
epoch: 25 step: 36 loss: 0.6931471824645996
epoch: 25 step: 37 loss: 0.6931471824645996
epoch: 25 step: 38 loss: 0.6931471824645996
epoch: 25 step: 39 loss: 0.6931471824645996
epoch: 25 step: 40 loss: 0.6931471824645996
epoch: 25 step: 41 loss: 0.69314

In [ ]:
torch.save(new_model.state_dict(), 'models/30epochstrainedmodel.pt')

In [ ]:
saved_model = ResNet()
saved_model.load_state_dict(torch.load('models/30epochstrainedmodel.pt'))

In [ ]:
saved_model.eval()
saved_model.cuda()

In [ ]:
testlossarr = []
#loss = torch.zeros(1, dtype=torch.double)
#loss.cuda()
avgloss = 0
for step, i in enumerate(test_dl):
    img, ohev, label = i
    img = Variable(img).cuda()
    ohev = Variable(ohev.float()).cuda()
    pred = saved_model(img)
    loss = loss_func(pred, ohev)
    avgloss +=loss.item()
    testlossarr.append(loss.cpu().detach().numpy())
    print('step:', step+1, 'loss:', loss.item())
    #testlossarr.append()
print(avgloss/17)

In [ ]:
testlossarr = np.array(testlossarr)
np.mean(testlossarr)

In [ ]:
temp = next(iter(train_dl))
img0 = Variable(temp[0][3][None, :]).cuda()
pred = saved_model(img0)
pred.cpu().detach().numpy()

In [ ]:
from summary import summary

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary

In [ ]:
summary(saved_model, (1, 224, 224))